## Importing libraries

In [63]:
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import seaborn as sns
import warnings

warnings.filterwarnings("ignore")

# database libraries
import sqlite3 as s3
from sqlalchemy import create_engine

In [64]:
import os

for file in os.listdir("../input"):
    _ = os.stat(os.path.abspath(".././input/"+file)).st_size / 1024 / 1024
    print(f"File name : {file} and file size is {_:.2f} MB")

os.stat(os.path.abspath(".././input/Train.csv"))

File name : SampleSubmission.csv and file size is 78.72 MB
File name : Test.csv and file size is 2253.54 MB
File name : Train.csv and file size is 6917.87 MB


os.stat_result(st_mode=33206, st_ino=93, st_dev=428019990, st_nlink=0, st_uid=0, st_gid=0, st_size=7253917400, st_atime=1647212094, st_mtime=1374504622, st_ctime=1647212094)

<p style = "color: red"> We have two files in the input for started, each one of them above 1 GB. In order to make the analysis easier to run on computer, <b>we will only be using the train file</b> for which we have the answers avaialable. For the test file, the predictions need to be submitted on kaggle, so it is of no use for the initial analysis</p>

## Reading input data and saving to a database

In [104]:
%cd ../input

g:\My Drive\Colab Notebooks\Keywork-Extraction-Kaggle-Stackoverflow\input


In [118]:
# Since the files are very big, it is not possible to load it in RAM and work with it
# We will make use of sqlite databases to run the operations

import sqlite3 as s3
from sqlalchemy import create_engine

prefix = lambda filename: "../input/"+filename

if not os.path.isfile(prefix("train.db")):
    # The database has not yet been initiated
    
    engine = create_engine("sqlite:///train.db")
    chunk = 180000   # 180k rows read at a time
    
    j = 0
    start_index = 1
    
    print("Reading data to sql database train.db\n" + "*"*30 + "\n")
    
    for data in pd.read_csv(prefix("Train.csv"), names=["Id", "Title", "Body", "Tags"], skiprows=1 \
        , chunksize=chunk, iterator=True, encoding="utf-8"):
        
        j+=1
        print(f"Rows read: {j*chunk}", end='\r')
        
        data.to_sql("data", engine, if_exists="append")
    

Reading data to sql database train.db
******************************



<p style = "color: red"> 
Here are few key observations:
<ol style = "color: red">
    <li>There are over 6 million questions in the training datasets</li>
    <li>Time taken to read the input data was over 30 minutes</li>
    <li>Data read is stored in a database instead of storing it in memory</li>
</ol>
</p>

In [154]:
with s3.connect("train.db") as con:
    x = pd.read_sql_query("""
                          SELECT Title, Body, Tags
                          FROM data 
                          LIMIT 100 
                          """, con)
    
x.head()

,Title,Body,Tags
0,How to check if an uploaded file is an image w...,<p>I'd like to check if an uploaded file is an...,php image-processing file-upload upload mime-t...
1,How can I prevent firefox from closing when I ...,"<p>In my favorite editor (vim), I regularly us...",firefox
2,R Error Invalid type (list) for variable,<p>I am import matlab file and construct a dat...,r matlab machine-learning
3,How do I replace special characters in a URL?,"<p>This is probably very simple, but I simply ...",c# url encoding
4,How to modify whois contact details?,<pre><code>function modify(.......)\n{\n $mco...,php api file-get-contents


As we can wee, there are 3 key columns:
* Title - this is the title of the question posted on SO
* Body - contains the body including the code snippets etc
* Tags - the tags that we want to predict for new questions

In [177]:
import IPython
import ipywidgets as widgets

@widgets.interact(i = (0, 10))
def fun(i):
    return IPython.display.HTML(f"<h3>Label</h3><h4><code>{x.Tags[i].split()}</code></h4><hr>" + f"<h2>{x.Title[i]}</h2>" + x.Body[i])
    

interactive(children=(IntSlider(value=5, description='i', max=10), Output()), _dom_classes=('widget-interact',…

### Count the total number of rows

In [126]:
with s3.connect("train.db") as con:
    x = pd.read_sql_query("""
                          SELECT count(*) 
                          FROM data 
                          """, con)
    
x

,count(*)
0,6034195
